In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用Vertex AI Tabular Workflows训练BigQuery ML ARIMA_PLUS模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabular_workflows/bqml_arima_plus.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/tabular_workflows/bqml_arima_plus.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      查看GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/tabular_workflows/bqml_arima_plus.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

在本教程中，您将扮演一位商店规划师的角色，必须确定他们将需要为2019年11月的每个产品和商店订购多少库存。您可以通过训练一个BigQuery ML（BQML）[ARIMA_PLUS](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create-time-series)预测模型来实现这一目标，使用历史销售数据。如果您需要执行多个模型训练的快速迭代，或者需要一个廉价的基准来衡量其他模型，那么BQML ARIMA_PLUS模型将会很有用。

了解更多关于[BQML ARIMA+用于表格数据的预测](https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting-arima/overview)。

### 目标

在本教程中，您将学习如何使用[Google Cloud Pipeline Components](https://cloud.google.com/vertex-ai/docs/pipelines/components-introduction)（GCPC）中的训练[Vertex AI Pipeline](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)来创建一个 BigQuery ML ARIMA_PLUS 模型，然后使用相应的预测管道进行批量预测。然后，您将使用相同数据训练一个 Vertex AI 预测模型，并比较评估指标。

本教程使用以下 Google Cloud ML 服务和资源：

- BigQuery
- Vertex AI

执行的步骤包括：

- 训练 BigQuery ML ARIMA_PLUS 模型。
- 查看 BigQuery ML 模型评估。
- 使用 BigQuery ML 模型进行批量预测。

### 数据集

为了展示使用BigQuery ML和Vertex AI Forecasting之间的权衡，本教程将使用一个合成数据集，其中产品销售依赖于诸多因素，如广告、假期和位置。您可以看到像ARIMA_PLUS这样的单变量模型如何在不明确了解这些因素信息的情况下预测未来销售，以及当了解这些因素时，像Vertex AI Forecasting这样的多变量模型将如何执行。

###成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage
* BigQuery / BigQuery ML

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)、[Cloud Storage定价](https://cloud.google.com/storage/pricing)和[BigQuery定价](https://cloud.google.com/bigquery/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

安装以下所需的软件包以执行此笔记本。

In [ ]:
! (pip3 install --upgrade --quiet \
    google-cloud-aiplatform==1.40.0 \
    google-cloud-bigquery[pandas]==3.17.1 \
    google-cloud-pipeline-components==2.9.0)

### 仅Colab使用：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

地区

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关[Vertex AI 地区](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"
DATA_REGION = "US"

### 验证您的 Google Cloud 账户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 不需要做任何操作，因为您已经通过验证。

**2. 本地 JupyterLab 实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，去除注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

服务账号或其他
* 请参考如何向您的服务账号授予云存储权限的方法，网址为：https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储桶

创建一个存储桶来存储诸如数据集等中间产物。

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时：运行下面的单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

服务账户

您可以使用服务账户来创建Vertex AI管道作业。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

在Vertex AI Pipelines中设置服务账号访问权限

运行以下命令，将您的服务账号访问权限授予读取和写入之前步骤中创建的存储桶中的管道工件。您只需要针对每个服务账号运行此步骤一次。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

导入库并定义常数。

In [ ]:
import json
import os
import urllib
import uuid

from google.cloud import aiplatform, bigquery
from google_cloud_pipeline_components.v1.automl.forecasting import utils

初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

定义训练和预测数据

创建一个BigQuery数据集

In [ ]:
arima_dataset_name = "forecasting_demo_arima"
arima_dataset_path = ".".join([PROJECT_ID, arima_dataset_name])

# Must be same region as TRAINING_DATASET_BQ_PATH.
client = bigquery.Client(project=PROJECT_ID)
bq_dataset_pre = bigquery.Dataset(arima_dataset_path)
bq_dataset_pre.location = DATA_REGION
try:
    bq_dataset = client.create_dataset(bq_dataset_pre)
except:
    bq_dataset = client.get_dataset(bq_dataset_pre)
print(f"Created bigquery dataset {arima_dataset_path} in {DATA_REGION}")

### 在BigQuery中准备训练数据

在训练模型之前，您必须首先生成我们的店铺销售数据集。这个数据集将包括多个产品和店铺，并且还将模拟因素如广告和假期效应。数据将被拆分为`TRAIN`、`VALIDATE`、`TEST`和`PREDICT`集，其中最后三个集都为1个月。

#### 先定义将创建这个基本销售数据的子查询。

In [ ]:
base_data_query = """
  WITH 

    -- Create time series for each product + store with some covariates.
    time_series AS (
      SELECT
        CONCAT("id_", store_id, "_", product_id) AS id,
        CONCAT('store_', store_id) AS store,
        CONCAT('product_', product_id) AS product,
        date,
        -- Advertise 1/100 products.
        IF(
          ABS(MOD(FARM_FINGERPRINT(CONCAT(product_id, date)), 100)) = 0,
          1,
          0
        ) AS advertisement,
        -- Mark Thanksgiving sales as holiday sales.
        IF(
          EXTRACT(DAYOFWEEK FROM date) = 6
            AND EXTRACT(MONTH FROM date) = 11
            AND EXTRACT(DAY FROM date) BETWEEN 23 AND 29,
          1,
          0
        ) AS holiday,
        -- Set when each data split ends.
        CASE
          WHEN date < '2019-09-01' THEN 'TRAIN'
          WHEN date < '2019-10-01' THEN 'VALIDATE'
          WHEN date < '2019-11-01' THEN 'TEST'
          ELSE 'PREDICT'
        END AS split,
      -- Generate the sales with one SKU per date.
      FROM
        UNNEST(GENERATE_DATE_ARRAY('2017-01-01', '2019-12-01')) AS date
      CROSS JOIN
        UNNEST(GENERATE_ARRAY(0, 10)) AS product_id
      CROSS JOIN
        UNNEST(GENERATE_ARRAY(0, 3)) AS store_id  
    ),
    
    -- Randomly determine factors that contribute to how syntheic sales are calculated. 
    time_series_sales_factors AS (
      SELECT
        *,
        ABS(MOD(FARM_FINGERPRINT(product), 10)) AS product_factor,
        ABS(MOD(FARM_FINGERPRINT(store), 10)) AS store_factor,
        [1.6, 0.6, 0.8, 1.0, 1.2, 1.8, 2.0][
          ORDINAL(EXTRACT(DAYOFWEEK FROM date))] AS day_of_week_factor,
        1 +  SIN(EXTRACT(MONTH FROM date) * 2.0 * 3.14 / 24.0) AS month_factor,    
        -- Advertised products have increased sales factors for 5 days.
        CASE
          WHEN LAG(advertisement, 0) OVER w = 1.0 THEN 1.2
          WHEN LAG(advertisement, 1) OVER w = 1.0 THEN 1.8
          WHEN LAG(advertisement, 2) OVER w = 1.0 THEN 2.4
          WHEN LAG(advertisement, 3) OVER w = 1.0 THEN 3.0
          WHEN LAG(advertisement, 4) OVER w = 1.0 THEN 1.4
          ELSE 1.0
        END AS advertisement_factor,
        IF(holiday = 1.0, 2.0, 1.0) AS holiday_factor,
        0.001 * ABS(MOD(FARM_FINGERPRINT(CONCAT(product, store, date)), 100)) AS noise_factor
      FROM
        time_series
      WINDOW w AS (PARTITION BY id ORDER BY date)
    ),
  
    -- Use factors to calculate synthetic sales for each time series. 
    base_data AS (
      SELECT
        id,
        store,
        product,
        date,
        split,
        advertisement,
        holiday,
        (
          (1 + store_factor) 
          * (1 + product_factor) 
          * (1 + month_factor + day_of_week_factor) 
          * (
            1.0 
            + 2.0 * advertisement_factor 
            + 3.0 * holiday_factor 
            + 5.0 * noise_factor
          )
        ) AS sales
      FROM
        time_series_sales_factors
      )
"""

接下来，将这些基本销售数据转换为您用于训练模型的数据集，以及在提供服务时传递给经过训练的模型的数据集。训练数据集将包括“TRAIN”、“VALIDATE”和“TEST”拆分，而预测数据集将包括“PREDICT”拆分，同时还包括“TEST”拆分以提供上下文信息。

In [ ]:
TRAINING_DATASET_BQ_PATH = f"{arima_dataset_path}.train"
PREDICTION_DATASET_BQ_PATH = f"{arima_dataset_path}.pred"

train_query = f"""
    CREATE OR REPLACE TABLE `{arima_dataset_path}.train` AS
    {base_data_query}
    SELECT *
    FROM base_data
    WHERE split != 'PREDICT'
"""
client.query(train_query).result()
print(f"Created {TRAINING_DATASET_BQ_PATH}.")

pred_query = f"""
    CREATE OR REPLACE TABLE `{arima_dataset_path}.pred` AS
    {base_data_query}
    SELECT *
    FROM base_data
    WHERE split = 'TEST'

    UNION ALL

    SELECT * EXCEPT (sales), NULL AS sales
    FROM base_data
    WHERE split = 'PREDICT'
"""
client.query(pred_query).result()
print(f"Created {PREDICTION_DATASET_BQ_PATH}.")

您可以查看生成的销售数据。在本教程的后续部分中，我们将展示时间序列以及我们的预测。

该模型使用从2017年1月到2019年10月的数据进行训练。 

#### 查看训练数据

In [ ]:
query = f"SELECT * FROM `{arima_dataset_path}.train` LIMIT 10"
client.query(query).to_dataframe().head()

用于预测的表格包含2019年11月的数据。它还包括2019年10月的实际数据作为背景信息。

#### 查看预测数据

In [ ]:
query = f"SELECT * FROM `{arima_dataset_path}.pred` LIMIT 10"
client.query(query).to_dataframe().head()

创建一个BigQuery ML ARIMA_PLUS模型

现在，您可以开始创建自己的BigQuery ML ARIMA_PLUS模型了。

与Vertex AI预测一样，您运行的流水线将使用训练和验证集训练评估模型，并使用回测在测试集上创建评估指标。最后，将生成一个服务模型，该模型使用所有可用数据。

**如何估算成本？**

回测涉及为测试集中的每个周期训练单个BigQuery ML模型，因此成本是测试集长度的函数，该长度是由窗口策略进行任何下采样后的结果。成本还会乘以训练的候选模型数量，这由`max_order`确定。

根据[BQ定价](https://cloud.google.com/bigquery-ml/pricing)，BigQuery ML模型创建成本为每TB 250美元。我们将使用最大阶数为3，这在存在多个时间序列时等于20个候选模型。我们的演示数据集大小为3 MB，并包括31个测试周期。我们的窗口策略的步长为1，因此所有周期都用于评估。

在本教程中，流水线的模型创建阶段成本为`3 MB * ($250 / 1024^2) * (31 / 1)周期 * 20个候选模型 = $0.44`。

## 创建并运行训练作业
使用ARIMA管道训练模型需要执行两个步骤：

1. 从GCPC下载训练管道。
2. 运行作业。

#### 创建训练作业

训练管道需要以下参数：

- `bigquery_destination_uri`: （可选）BigQuery数据集URI。用于导出指标表和模型。如果未提供，将为用户创建一个。
- `data_granularity_unit`: 用于指定时间粒度（小时，天，周，月等）的枚举。
- `data_source_csv_filenames`或`data_source_bigquery_table_path`: 分别为存储在GCR中的CSV文件或BigQuery表的URI。
- `evaluated_examples_destination_uri`: （可选）BigQuery数据集URI或表URI。用于导出评估示例表。如果未提供，将使用bigquery_destination_uri。
- `forecast_horizon`: 预测的周期数。
- 数据拆分策略为：
   - `predefined_split_key`: 包含`TRAIN`、`VALIDATE`或`TEST`的列，用于表示每行的拆分。
   - `training_fraction`、`validation_fraction`和`test_fraction`用于设置在时间列上按时间顺序拆分的比例。
   - `timestamp_split_key`加上先前选项中的比例，用于在其它列上执行分数拆分。
- 窗口策略为：
   - `window_column`: 一个布尔列，决定是否考虑每行在计算评估指标时。
   - `window_stride_length`: 每N行将用于计算评估指标。
   - `window_max_count`: 降序行，只有给定数量的行用于计算评估指标。
- `target_column`: 目标列的名称。
- `time_column`: 时间列的名称。
- `time_series_identifier_column`: id列的名称。
- `max_order`: 介于1到5之间的整数，表示ARIMA_PLUS参数搜索空间的大小。5会得到最高精度的模型，但训练时间/成本也会最长。

有关完整参数列表，请参阅GCPC SDK[文档](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.9.0/api/v1/automl/forecasting.html#v1.automl.forecasting.get_bqml_arima_train_pipeline_and_parameters)。

训练管道的执行可能需要约**20分钟**。

In [ ]:
time_column = "date"  # @param {type: "string"}
time_series_identifier_column = "id"  # @param {type: "string"}
target_column = "sales"  # @param {type: "string"}
forecast_horizon = 30  # @param {type: "integer"}
data_granularity_unit = "day"  # @param {type: "string"}
split_column = "split"  # @param {type: "string"}
window_stride_length = 1  # @param {type: "integer"}
max_order = 3  # @param {type: "integer"}
override_destination = True  # @param {type: "boolean"}

(
    train_job_spec_path,
    train_parameter_values,
) = utils.get_bqml_arima_train_pipeline_and_parameters(
    project=PROJECT_ID,
    location=REGION,
    root_dir=os.path.join(BUCKET_URI, "pipeline_root"),
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    target_column=target_column,
    forecast_horizon=forecast_horizon,
    data_granularity_unit=data_granularity_unit,
    predefined_split_key=split_column,
    data_source_bigquery_table_path=TRAINING_DATASET_BQ_PATH,
    window_stride_length=window_stride_length,
    bigquery_destination_uri=arima_dataset_path,
    override_destination=override_destination,
    max_order=max_order,
)

运行培训管道

使用Vertex AI Python SDK启动培训管道运行。一旦运行开始，以下单元格会输出一个链接，让您可以监视运行。链接应如下所示： 

`https://console.cloud.google.com/vertex-ai/locations/[REGION]/pipelines/runs/[DISPLAY_NAME]`

In [ ]:
# The display name should be unique even if this cell is rerun.
DISPLAY_NAME = f"forecasting-demo-train-{str(uuid.uuid1())}"

job = aiplatform.PipelineJob(
    job_id=DISPLAY_NAME,
    display_name=DISPLAY_NAME,
    pipeline_root=os.path.join(BUCKET_URI, DISPLAY_NAME),
    template_path=train_job_spec_path,
    parameter_values=train_parameter_values,
    enable_caching=False,
)
job.run(service_account=SERVICE_ACCOUNT)

## 查看模型评估分数
在模型训练完成后，您可以查看它的评估分数。

#### 评估指标始终通过目标数据集中的`metrics`表报告。

In [ ]:
for task_detail in job.gca_resource.job_detail.task_details:
    if task_detail.task_name == "create-metrics-artifact":
        metrics = task_detail.outputs["evaluation_metrics"].artifacts[0].metadata
        break
else:
    raise ValueError("Couldn't find the model evaluation task.")

print("Evaluation metrics:\n")
dict(metrics)

如果您想要计算自己的评估指标，您可以查看用于计算评估指标的预测。

#### 查看用于计算评估指标的预测

包含所有这些预测的表格称为“评估示例”。在这个表格中，每个不同的“predicted_on_date”代表一个预测窗口的开始期。回测指标利用所有这些窗口。

In [ ]:
query = f"SELECT * FROM `{arima_dataset_path}.evaluated_examples`"
arima_examples = client.query(query).to_dataframe()
arima_examples.head()

## 创建和运行预测作业

### 创建预测作业
现在您的 Model 资源已经训练好，您可以使用预测流水线进行批量预测，以下是预测作业的参数：

- `bigquery_destination_uri`: （可选）BigQuery 数据集 URI。用于导出指标表和模型。如果未提供，则我们将为用户创建一个。
- `data_source_csv_filenames` 或 `data_source_bigquery_table_path`: 分别用于存储在 GCR 中的 CSV 或 BigQuery 表的 URI。
- `generate_explanation`: 如果为 True，预测结果表将具有一些额外的 xAI 列。
- `model_name`: 要用于预测的现有 BigQuery ML ARIMA_PLUS 模型的名称。

有关完整参数列表，请查看 GCPC SDK [文档](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-2.9.0/api/v1/automl/forecasting.html#v1.automl.forecasting.get_bqml_arima_predict_pipeline_and_parameters)。

执行预测流水线可能需要大约 **5 分钟**。

In [ ]:
# Get the model name programmatically, you can find this by looking at the
# execution graph in Vertex AI Pipelines.
for task_detail in job.gca_resource.job_detail.task_details:
    if task_detail.task_name == "bigquery-create-model-job":
        model_name = task_detail.outputs["model"].artifacts[0].metadata["modelId"]
        break
else:
    raise ValueError("Couldn't find the model training task.")


(
    predict_job_spec_path,
    predict_parameter_values,
) = utils.get_bqml_arima_predict_pipeline_and_parameters(
    project=PROJECT_ID,
    location=REGION,
    model_name=f"{arima_dataset_path}.{model_name}",
    data_source_bigquery_table_path=PREDICTION_DATASET_BQ_PATH,
    bigquery_destination_uri=arima_dataset_path,
)

运行预测流水线

使用Vertex AI Python SDK启动预测流水线运行。一旦运行开始，下面的单元格将输出一个链接，允许您监视运行。链接应该看起来像这样：

`https://console.cloud.google.com/vertex-ai/locations/[REGION]/pipelines/runs/[DISPLAY_NAME]`

In [ ]:
# The display name should be unique even if this cell is rerun.
DISPLAY_NAME = f"forecasting-demo-predict-{str(uuid.uuid1())}"


job = aiplatform.PipelineJob(
    job_id=DISPLAY_NAME,
    display_name=DISPLAY_NAME,
    pipeline_root=os.path.join(BUCKET_URI, DISPLAY_NAME),
    template_path=predict_job_spec_path,
    parameter_values=predict_parameter_values,
    enable_caching=False,
)
job.run(service_account=SERVICE_ACCOUNT)

获取预测结果

接下来，从已完成的批量预测作业中获取结果。这些结果通常会被写入输出数据集中名为`predictions`的表中。

In [ ]:
# Get the prediction table programmatically, you can find this by looking at the
# execution graph in Vertex AI Pipelines.
for task_detail in job.gca_resource.job_detail.task_details:
    if task_detail.task_name == "bigquery-query-job":
        pred_table = (
            task_detail.outputs["destination_table"].artifacts[0].metadata["tableId"]
        )
        break
else:
    raise ValueError("Couldn't find the prediction task.")

query = f"SELECT * FROM `{arima_dataset_path}.{pred_table}`"
arima_preds = client.query(query).to_dataframe()
arima_preds.head()

## 可视化预测

最后，点击以下链接在[数据工作室](https://support.google.com/datastudio/answer/6283323?hl=en)中可视化生成的预测结果。
本节中的代码块动态生成一个数据工作室链接，指定了模板、预测位置和生成图表的查询。数据是从之前生成的预测中填充的。

您可以在https://datastudio.google.com/c/u/0/reporting/067f70d2-8cd6-4a4c-a099-292acd1053e8 查看使用的模板。这是谷歌专门创建的用于查看预测预测的模板。

In [ ]:
def _sanitize_bq_uri(bq_uri: str):
    if bq_uri.startswith("bq://"):
        bq_uri = bq_uri[5:]
    return bq_uri.replace(":", ".")


def get_data_studio_link(
    batch_prediction_bq_input_uri: str,
    batch_prediction_bq_output_uri: str,
    time_column: str,
    time_series_identifier_column: str,
    target_column: str,
):
    """Creates a link that fills in the demo Data Studio template."""
    batch_prediction_bq_input_uri = _sanitize_bq_uri(batch_prediction_bq_input_uri)
    batch_prediction_bq_output_uri = _sanitize_bq_uri(batch_prediction_bq_output_uri)
    query = f"""
        SELECT
          CAST(input.{time_column} as DATETIME) timestamp_col,
          CAST(input.{time_series_identifier_column} as STRING) time_series_identifier_col,
          CAST(input.{target_column} as NUMERIC) historical_values,
          CAST(predicted_{target_column}.value as NUMERIC) predicted_values,
        FROM `{batch_prediction_bq_input_uri}` input
        LEFT JOIN `{batch_prediction_bq_output_uri}` output
          ON
            TIMESTAMP(input.{time_column}) = TIMESTAMP(output.{time_column})
            AND CAST(input.{time_series_identifier_column} as STRING) = CAST(
              output.{time_series_identifier_column} as STRING)
    """
    params = {
        "templateId": "067f70d2-8cd6-4a4c-a099-292acd1053e8",
        "ds0.connector": "BIG_QUERY",
        "ds0.projectId": PROJECT_ID,
        "ds0.billingProjectId": PROJECT_ID,
        "ds0.type": "CUSTOM_QUERY",
        "ds0.sql": query,
    }
    base_url = "https://datastudio.google.com/c/u/0/reporting"
    url_params = urllib.parse.urlencode({"params": json.dumps(params)})
    return f"{base_url}?{url_params}"

In [ ]:
actuals_table = f"{arima_dataset_path}.actuals"
query = f"""
    CREATE OR REPLACE TABLE `{actuals_table}` AS
    {base_data_query}
    SELECT *
    FROM base_data
    WHERE split != 'TRAIN'
"""
client.query(query).result()
print(f"Created {actuals_table}.")

In [ ]:
print("Click the link below to view ARIMA predictions:")
print(
    get_data_studio_link(
        batch_prediction_bq_input_uri=actuals_table,
        batch_prediction_bq_output_uri=f"{arima_dataset_path}.{pred_table}",
        time_column=time_column,
        time_series_identifier_column=time_series_identifier_column,
        target_column=target_column,
    )
)

清理 Vertex AI 和 BigQuery 资源

要清理此项目中使用的所有 Google Cloud 资源，您可以删除用于本教程的 Google Cloud 项目。

否则，您可以删除此教程中创建的各个资源：

- Cloud Storage 存储桶
- BigQuery 表

In [ ]:
# Delete output datasets
client.delete_dataset(arima_dataset_path, delete_contents=True, not_found_ok=True)

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI